In [24]:
import pandas as pd
import xml.etree.ElementTree as ET
import io
import datetime

In [25]:
etree = ET.parse('../../data/raw/bc3/corpus.xml')

In [26]:
root = etree.getroot()

In [27]:
message_cols = ["Received", "From", "To", "Subject", "Text", "thread"]
thread_cols = ["name", "listno"]
message_df = pd.DataFrame(columns=message_cols);
thread_df = pd.DataFrame(columns=thread_cols);

In [28]:
def get_text(node):
    message = ''
    for sent in node.findall('Sent'):
        message += sent.text + '\n'
    return message

In [29]:
for thread in root.findall('thread'):
    thread_name = thread.find('name').text
    thread_listno = thread.find('listno').text
    thread_df = thread_df.append({'name': thread_name, 'listno': thread_listno}, ignore_index=True)
    
    for message in thread.findall('DOC'):
        message_received = message.find('Received').text
        message_from = message.find('From').text
        message_to = message.find('To').text
        message_subject = message.find('Subject').text
        message_text = [get_text(message.find('Text')).strip('\n')]
        message_text = []
        for i in message.find('Text'):
            message_text.append({'type': i.tag, 'id': i.get('id'), 'value': i.text})
        message_thread = thread_listno
        message_threadname = thread_name
        message_df = message_df.append(
            {
                'Received': message_received,
                'From': message_from,
                'To': message_to,
                'Subject': message_subject,
                'Text': message_text,
                'thread': message_thread,
                'thread_name': message_threadname
            },
            ignore_index=True)

In [30]:
message_df.head()

,Received,From,To,Subject,Text,thread,thread_name
0,Tue Dec 08 07:30:52 -0800 1998,Jacob Palme <jpalme@dsv.su.se>,discuss@apps.ietf.org,Extending IETF meetings to two weeks?,"[{'type': 'Sent', 'id': '1.1', 'value': 'The I...",007-7484738,Extending IETF meetings to two weeks?
1,Wed Dec 09 20:21:11 -0800 1998,Terry Allen <tallen@sonic.net>,"discuss@apps.ietf.org,jpalme@dsv.su.se",Re: Extending IETF meetings to two weeks?,"[{'type': 'Sent', 'id': '2.1', 'value': '> The...",007-7484738,Extending IETF meetings to two weeks?
2,Thu Dec 10 07:09:32 -0800 1998,Brian E Carpenter <brian@hursley.ibm.com>,Terry Allen <tallen@sonic.net>,Re: Extending IETF meetings to two weeks?,"[{'type': 'Sent', 'id': '3.1', 'value': 'Terry...",007-7484738,Extending IETF meetings to two weeks?
3,Thu Dec 10 11:55:56 -0800 1998,Larry Masinter <masinter@parc.xerox.com>,"<agenda@ietf.org> , <discuss@apps.ietf.org>",create 'final' IETF agenda schedule earlier?,"[{'type': 'Sent', 'id': '4.1', 'value': 'Worki...",007-7484738,Extending IETF meetings to two weeks?
4,Fri Dec 11 16:02:06 -0800 1998,Richard Shockey <rshockey@ix.netcom.com>,discuss@apps.ietf.org,Re: create 'final' IETF agenda schedule earlier?,"[{'type': 'Sent', 'id': '5.1', 'value': '> For...",007-7484738,Extending IETF meetings to two weeks?


In [32]:
thread_df.head()

,name,listno
0,Extending IETF meetings to two weeks?,007-7484738
1,SWADEurope postcard picture,015-2625401
2,CHI 2003?,023-2964247
3,September F2F,026-4380722
4,Location of August 30-31 1999 Face to Face,043-10248963


In [33]:
message_df['Received'] = pd.to_datetime(message_df['Received'], infer_datetime_format=True)

In [34]:
message_df.head()

,Received,From,To,Subject,Text,thread,thread_name
0,1998-12-08 07:30:52-08:00,Jacob Palme <jpalme@dsv.su.se>,discuss@apps.ietf.org,Extending IETF meetings to two weeks?,"[{'type': 'Sent', 'id': '1.1', 'value': 'The I...",007-7484738,Extending IETF meetings to two weeks?
1,1998-12-09 20:21:11-08:00,Terry Allen <tallen@sonic.net>,"discuss@apps.ietf.org,jpalme@dsv.su.se",Re: Extending IETF meetings to two weeks?,"[{'type': 'Sent', 'id': '2.1', 'value': '> The...",007-7484738,Extending IETF meetings to two weeks?
2,1998-12-10 07:09:32-08:00,Brian E Carpenter <brian@hursley.ibm.com>,Terry Allen <tallen@sonic.net>,Re: Extending IETF meetings to two weeks?,"[{'type': 'Sent', 'id': '3.1', 'value': 'Terry...",007-7484738,Extending IETF meetings to two weeks?
3,1998-12-10 11:55:56-08:00,Larry Masinter <masinter@parc.xerox.com>,"<agenda@ietf.org> , <discuss@apps.ietf.org>",create 'final' IETF agenda schedule earlier?,"[{'type': 'Sent', 'id': '4.1', 'value': 'Worki...",007-7484738,Extending IETF meetings to two weeks?
4,1998-12-11 16:02:06-08:00,Richard Shockey <rshockey@ix.netcom.com>,discuss@apps.ietf.org,Re: create 'final' IETF agenda schedule earlier?,"[{'type': 'Sent', 'id': '5.1', 'value': '> For...",007-7484738,Extending IETF meetings to two weeks?


In [35]:
date_data = {
    'mail_sent_day': [],
    'mail_sent_date': [],
    'mail_sent_time': [],
    'mail_sent_year': []
}
tags = { '1' : 'Monday' ,'2' : 'Tuesday' ,'3' : 'Wednesday' ,'4' : 'Thursday' ,'5' : 'Friday' ,'6' : 'Saturday' ,'7' : 'Sunday' }

for timestamp in message_df.Received:
    date_data['mail_sent_day'].append(tags[str(timestamp.isoweekday())])
    date_data['mail_sent_date'].append(timestamp.date())
    date_data['mail_sent_time'].append(timestamp.time())
    date_data['mail_sent_year'].append(timestamp.year)

date_df = pd.DataFrame.from_dict(date_data, orient='columns')
date_df.index = message_df.index
    
message_df = pd.concat([message_df, date_df], axis=1)
del date_df, date_data, tags

In [36]:
message_df.head()

,Received,From,To,Subject,Text,thread,thread_name,mail_sent_day,mail_sent_date,mail_sent_time,mail_sent_year
0,1998-12-08 07:30:52-08:00,Jacob Palme <jpalme@dsv.su.se>,discuss@apps.ietf.org,Extending IETF meetings to two weeks?,"[{'type': 'Sent', 'id': '1.1', 'value': 'The I...",007-7484738,Extending IETF meetings to two weeks?,Tuesday,1998-12-08,07:30:52,1998
1,1998-12-09 20:21:11-08:00,Terry Allen <tallen@sonic.net>,"discuss@apps.ietf.org,jpalme@dsv.su.se",Re: Extending IETF meetings to two weeks?,"[{'type': 'Sent', 'id': '2.1', 'value': '> The...",007-7484738,Extending IETF meetings to two weeks?,Wednesday,1998-12-09,20:21:11,1998
2,1998-12-10 07:09:32-08:00,Brian E Carpenter <brian@hursley.ibm.com>,Terry Allen <tallen@sonic.net>,Re: Extending IETF meetings to two weeks?,"[{'type': 'Sent', 'id': '3.1', 'value': 'Terry...",007-7484738,Extending IETF meetings to two weeks?,Thursday,1998-12-10,07:09:32,1998
3,1998-12-10 11:55:56-08:00,Larry Masinter <masinter@parc.xerox.com>,"<agenda@ietf.org> , <discuss@apps.ietf.org>",create 'final' IETF agenda schedule earlier?,"[{'type': 'Sent', 'id': '4.1', 'value': 'Worki...",007-7484738,Extending IETF meetings to two weeks?,Thursday,1998-12-10,11:55:56,1998
4,1998-12-11 16:02:06-08:00,Richard Shockey <rshockey@ix.netcom.com>,discuss@apps.ietf.org,Re: create 'final' IETF agenda schedule earlier?,"[{'type': 'Sent', 'id': '5.1', 'value': '> For...",007-7484738,Extending IETF meetings to two weeks?,Friday,1998-12-11,16:02:06,1998


In [37]:
thread_df = thread_df.set_index('listno')

In [38]:
thread_df.head()

,name
listno,
007-7484738,Extending IETF meetings to two weeks?
015-2625401,SWADEurope postcard picture
023-2964247,CHI 2003?
026-4380722,September F2F
043-10248963,Location of August 30-31 1999 Face to Face


In [39]:
thread_data = {
    'count': [],
    'start_date': [],
    'end_date': [],
    'participants': []
}
for thread in thread_df.iterrows():
    messages = message_df.loc[message_df['thread'] == thread[0]]
    thread_data['count'].append(messages.count()[0])
    thread_data['start_date'].append(min(messages.Received))
    thread_data['end_date'].append(max(messages.Received))
    participants = []
    for message in messages.iterrows():
        if message[1].To is not None:
            participants += ([message[1].From] + message[1].To.split(','))
        else:
            participants += [message[1].From]
    thread_data['participants'].append(list(set(participants)))
    
thread_data_df = pd.DataFrame.from_dict(thread_data, orient='columns')
thread_data_df.head()
thread_data_df.index = thread_df.index
    
thread_df = pd.concat([thread_df, thread_data_df], axis=1)
del thread_data_df, thread_data

In [40]:
thread_df.head()

,name,count,start_date,end_date,participants
listno,,,,,
007-7484738,Extending IETF meetings to two weeks?,5,1998-12-08 07:30:52-08:00,1998-12-11 16:02:06-08:00,"[ <discuss@apps.ietf.org>, Jacob Palme <jpalme..."
015-2625401,SWADEurope postcard picture,6,2003-04-25 06:39:28-07:00,2003-04-25 15:18:38-07:00,"[public-esw@w3.org, ""'Libby Miller'"" <Libby.Mi..."
023-2964247,CHI 2003?,7,2003-03-26 00:46:07-08:00,2003-04-03 12:02:24-08:00,"[Chip Alexander <Chip.Alexander@sun.com>, Keit..."
026-4380722,September F2F,5,2003-05-30 12:15:41-07:00,2003-06-04 19:35:33-07:00,"[Monica J. Martin <monica.martin@sun.com>, Fra..."
043-10248963,Location of August 30-31 1999 Face to Face,8,1999-07-15 19:40:16-07:00,1999-07-19 11:55:20-07:00,[Barb Fox (Exchange) <bfox@Exchange.Microsoft....


In [41]:
message_df.to_pickle('bc3_mails.pkl')
thread_df.to_pickle('bc3_threads.pkl')

In [42]:
etree = ET.parse('../../data/raw/bc3/annotation.xml')
root = etree.getroot()

In [43]:
annotation_cols = ["listno", "desc", "date", "summary", "sentences", "labels"]
annotation_df = pd.DataFrame(columns=annotation_cols);

In [44]:
for thread in root.findall('thread'):
    thread_listno = thread.find('listno').text
    
    for annotation in thread.findall('annotation'):
        desc = annotation.find('desc').text
        date = annotation.find('date').text

        summary = []
        sentences = []
        labels = []
        for i in annotation.find('summary'):
            summary.append({'type': i.tag, 'link': i.get('link'), 'value': i.text})
        for i in annotation.find('sentences'):
            sentences.append({'type': i.tag, 'id': i.get('id'), 'value': i.text})
        for i in annotation.find('labels'):
            labels.append({'type': i.tag, 'id': i.get('id'), 'value': i.text})
        annotation_df = annotation_df.append({
            'listno': thread_listno,
            'desc': desc,
            'date': date,
            'summary': summary,
            'sentences': sentences,
            'labels': labels
        }, ignore_index=True)


In [45]:
annotation_df.head()

,listno,desc,date,summary,sentences,labels
0,067-11978590,Annotator7-Part2,Fri May 23 14:01:53 -0700 2008,"[{'type': 'sent', 'link': '1.1', 'value': 'Wen...","[{'type': 'sent', 'id': '1.1', 'value': None},...","[{'type': 'prop', 'id': '1.2', 'value': None},..."
1,067-11978590,Annotator8-Part1,Tue May 20 13:44:26 -0700 2008,"[{'type': 'sent', 'link': '', 'value': 'This e...","[{'type': 'sent', 'id': '1.2', 'value': None},...","[{'type': 'prop', 'id': '6.16', 'value': None}..."
2,067-11978590,Annotator6-Part2,Tue May 20 10:26:40 -0700 2008,"[{'type': 'sent', 'link': '1.1', 'value': 'The...","[{'type': 'sent', 'id': '1.1', 'value': None},...","[{'type': 'meta', 'id': '2.3', 'value': None},..."
3,070-4154814,Annotator13-Part1,Wed Jun 04 13:43:36 -0700 2008,"[{'type': 'sent', 'link': '1.1', 'value': 'Ste...","[{'type': 'sent', 'id': '1.1', 'value': None},...","[{'type': 'meta', 'id': '3.4', 'value': None},..."
4,070-4154814,Annotator12-Part1,Wed Jun 04 13:37:52 -0700 2008,"[{'type': 'sent', 'link': '1.1', 'value': 'Ste...","[{'type': 'sent', 'id': '1.1', 'value': None},...","[{'type': 'req', 'id': '3.12', 'value': None},..."


In [46]:
annotation_df.to_pickle('bc3_annotation.pkl')